## Cats and Dogs
---
Done as a part of an Unstop Course, 'Image Based Recognition'. The model was trained of Kaggle's Cats and Dogs dataset with over 24000 images. I achieved an accuracy of 92% after 25 epochs.

### Importing Libraries

In [33]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

### Data Preprocessing

In [9]:
# using kagglehub to download the cats and dogs dataset, it has 12000 images for both cats and dogs
import kagglehub

# Download latest version
path = kagglehub.dataset_download("karakaggle/kaggle-cat-vs-dog-dataset")

print("Path to dataset files:", path)

100%|██████████| 787M/787M [00:26<00:00, 30.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/karakaggle/kaggle-cat-vs-dog-dataset/versions/1


In [29]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/karakaggle/kaggle-cat-vs-dog-dataset/versions/1/kagglecatsanddogs_3367a/PetImages"
print("Contents of the dataset directory:", os.listdir(dataset_path))

Contents of the dataset directory: ['Cat', 'Dog']


In [24]:
'''
ImageDataGenerator allows for real time image augmentation as we process the images for the model.
In other words, we can slightly alter each image using various parameters like shear, zoom, flip, brightness, sclaing, etc, to augment the data. We do this to avoid overfitting.
This saves memory.
`flow_from_directory` is how we apply it on the dataset directly. It automatically detects images and classes.
'''
train_data = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

train_set = train_data.flow_from_directory(
    dataset_path,
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 24959 images belonging to 2 classes.


In [28]:
test_data = ImageDataGenerator(rescale = 1./255)
test_set = test_data.flow_from_directory(
    dataset_path,
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 24959 images belonging to 2 classes.


### CNN
---
`Sequential()` is used for straightforward networks where we have one input and output tensors. We can stack layers linearly and simply using this class.
`add` we use this to add layers.
`Conv2D` applies a 2D convolution. We have 32 such filters and their size is 3. This is the default setup. Input shape is also correct in our case as our images are 64x64 and colored so 64x64x3.

In [25]:
cnn = tf.keras.models.Sequential() # init neural net

# first conv layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) # apply convolution after deciding activation function and filter size
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # apply pooling

# second conv layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# flattening
cnn.add(tf.keras.layers.Flatten())

# full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
# compile the cnn
'''
Adam gradient descent is one of the many gradient descent techniques. We can use other metrics as well but accuracy is good enough.
Since this is binary classification our loss reflects that.
'''
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Testing
---
`cnn.fit()` based on the number of epochs, the method fits the model to our dataset iteratively.

In [30]:
# testing the model
cnn.fit(x = train_set, validation_data = test_set, epochs = 25)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


204/780 ━━━━━━━━━━━━━━━━━━━━ 2:29 259ms/step - accuracy: 0.5508 - loss: 0.6918

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


780/780 ━━━━━━━━━━━━━━━━━━━━ 267s 338ms/step - accuracy: 0.6106 - loss: 0.6497 - val_accuracy: 0.7442 - val_loss: 0.5192
Epoch 2/25
780/780 ━━━━━━━━━━━━━━━━━━━━ 268s 343ms/step - accuracy: 0.7360 - loss: 0.5281 - val_accuracy: 0.7889 - val_loss: 0.4580
Epoch 3/25
780/780 ━━━━━━━━━━━━━━━━━━━━ 239s 306ms/step - accuracy: 0.7681 - loss: 0.4852 - val_accuracy: 0.8046 - val_loss: 0.4219
Epoch 4/25
780/780 ━━━━━━━━━━━━━━━━━━━━ 241s 308ms/step - accuracy: 0.7906 - loss: 0.4504 - val_accuracy: 0.8130 - val_loss: 0.4113
Epoch 5/25
780/780 ━━━━━━━━━━━━━━━━━━━━ 238s 304ms/step - accuracy: 0.8050 - loss: 0.4198 - val_accuracy: 0.8027 - val_loss: 0.4241
Epoch 6/25
780/780 ━━━━━━━━━━━━━━━━━━━━ 241s 309ms/step - accuracy: 0.8135 - loss: 0.4036 - val_accuracy: 0.8258 - val_loss: 0.3861
Epoch 7/25
780/780 ━━━━━━━━━━━━━━━━━━━━ 260s 305ms/step - accuracy: 0.8216 - loss: 0.3937 - val_accuracy: 0.8189 - val_loss: 0.3962
Epoch 8/25
780/780 ━━━━━━━━━━━━━━━━━━━━ 254s 325ms/step - accuracy: 0.8289 - loss: 0.37

In [35]:
# we can use the predict() method to test a single image
test_image = image.load_img('/content/10.jpg',target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = cnn.predict(test_image)
train_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
cat


The prediction is correct!!

## References
- Keras Documentation https://www.analyticsvidhya.com/blog/2020/08/image-augmentation-on-the-fly-using-keras-imagedatagenerator/
- Unstop course lectures https://unstop.com/practice/machine-learning-ai-project/image-based-recognition
- Dataset from kaggle https://www.kaggle.com/datasets/karakaggle/kaggle-cat-vs-dog-dataset
- StackOverflow https://stackoverflow.com/questions/59953248/how-can-i-predict-single-image-in-keras-cnn-model
- G4G https://www.geeksforgeeks.org/keras-fit-and-keras-fit_generator/